In [77]:
import pyspark
import os
import sys
#Import DB Infos
sys.path.insert(1, '/home/yankees/JOURNAL-OFFICIEL/spark-journal-officiel/commons')
import DatabaseConfig as dbcfg
import Utils
import pandas as pd
import pathlib


from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

'''
This function helps specify the database and table to load for the analysis
arguments:
    #database name
    #table name
return:
    #table_dataframe
'''
def load_Db_Infos(db,table):
    table_dataframe = spark.read \
    .format("jdbc") \
    .option("url", str(db)) \
    .option("dbtable", str(table)) \
    .option("user", dbcfg.DB_INFOS['JOURNAL_DB_USER']) \
    .option("password", dbcfg.DB_INFOS['JOURNAL_DB_PASSWORD']) \
    .option("driver", dbcfg.DB_INFOS['JOURNAL_DB_DRIVER']) \
    .load()
    return table_dataframe


'''
This function will make different category on specific table like 
Decret - Categories ("Ratification de l'accord de prêt", "Permis de recherche minière", "Nomination").
Finally, this function will generate the csv file corresponding to the category
arguments:
    #table_name
    #list_keyword
return:
    #dataframe filtered for each element keywoord
'''
def generate_request_dataset_decret(table_name,list_keyword): 
    #Generate a list based on the table
    table_list = load_Db_Infos(str(CIV),table_name)
    
    #Generate the dataFrame based on the list_keyword
    for keyword in list_keyword:
        df_list_keyword = table_list.filter(table_list['tag_decret'].contains(str(keyword)))
    
        #Create CSV file containing the decret
        path = os.path.dirname('/home/yankees/JOURNAL-OFFICIEL/spark-journal-officiel/Decrets/')
        keyword_file = os.path.join(path, '_'.join(str(keyword).split()) + '.csv')
        df_list_keyword.write.mode('overwrite').option('header', 'true').csv(keyword_file)
        
        #Print DataFrame Schema
        df_list_keyword.printSchema()
        
        #Print the DataFrame
        df_list_keyword.show()
    
   
    return df_list_keyword.count()
  
    

if __name__ == "__main__":
    print("Journal Officiel Data Analysis")
    
    #Load specific country database
    CIV = dbcfg.DB_INFOS['JOURNAL_DB_URL_CIV']

Journal Officiel Data Analysis


In [78]:
load_Db_Infos(CIV,"decret")

DataFrame[id: bigint, numdecret: string, annee: int, date_publication: date, categorie_decret: string, type_acte: string, decretintro: string, decret_texte_url: string, signature: string, journal_officiel_id: bigint, numero_journal_officiel: string, tag_decret: string, montantdecret: bigint, nom_institution: string, reference_pret_decret: string, localite_decret: string, datesubmit: date]

In [79]:
generate_request_dataset_decret("decret",["accord de prêt", "Nomination", "Permis de recherche minière"])

root
 |-- id: long (nullable = true)
 |-- numdecret: string (nullable = true)
 |-- annee: integer (nullable = true)
 |-- date_publication: date (nullable = true)
 |-- categorie_decret: string (nullable = true)
 |-- type_acte: string (nullable = true)
 |-- decretintro: string (nullable = true)
 |-- decret_texte_url: string (nullable = true)
 |-- signature: string (nullable = true)
 |-- journal_officiel_id: long (nullable = true)
 |-- numero_journal_officiel: string (nullable = true)
 |-- tag_decret: string (nullable = true)
 |-- montantdecret: long (nullable = true)
 |-- nom_institution: string (nullable = true)
 |-- reference_pret_decret: string (nullable = true)
 |-- localite_decret: string (nullable = true)
 |-- datesubmit: date (nullable = true)

+---+---------+-----+----------------+-----------------+-------------------+--------------------+--------------------+-----------------+-------------------+-----------------------+--------------------+-------------+--------------------+----

4